In [1]:
# 데이터 불러오기
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df = pd.read_csv("서울시 상권분석서비스(집객시설-행정동).csv", encoding="EUC-KR", encoding_errors="replace", low_memory=False)

print(df.head())
print(df.info())

   기준_년분기_코드    행정동_코드 행정동_코드_명  집객시설_수  관공서_수  은행_수  종합병원_수  일반_병원_수  약국_수  \
0      20241  11305603      번2동     110    5.0   NaN     NaN      NaN   7.0   
1      20234  11110515    청운효자동      93    6.0   3.0     NaN      NaN   4.0   
2      20233  11110515    청운효자동      93    6.0   3.0     NaN      NaN   4.0   
3      20241  11170590      용문동      54    1.0   NaN     NaN      NaN   3.0   
4      20241  11110515    청운효자동      93    6.0   3.0     NaN      NaN   4.0   

   유치원_수  ...  대학교_수  백화점_수  슈퍼마켓_수  극장_수  숙박_시설_수  공항_수  철도_역_수  버스_터미널_수  \
0    1.0  ...    NaN    NaN     NaN   NaN      1.0   NaN     NaN       NaN   
1    1.0  ...    8.0    NaN     NaN   NaN      NaN   NaN     NaN       NaN   
2    1.0  ...    8.0    NaN     NaN   NaN      NaN   NaN     NaN       NaN   
3    NaN  ...    NaN    NaN     NaN   NaN      NaN   NaN     NaN       NaN   
4    1.0  ...    8.0    NaN     NaN   NaN      NaN   NaN     NaN       NaN   

   지하철_역_수  버스_정거장_수  
0      NaN      54.0  
1      NaN

In [2]:
df.drop(["기준_년분기_코드","행정동_코드"], axis=1, inplace=True)

In [3]:
dong_df = pd.read_excel("dong_info.xlsx")
print(dong_df.head())

   Unnamed: 0         adm_nm
0           0  서울특별시 종로구 사직동
1           1  서울특별시 종로구 삼청동
2           2  서울특별시 종로구 부암동
3           3  서울특별시 종로구 평창동
4           4  서울특별시 종로구 무악동


In [4]:
df["행정동_코드_명"] = df["행정동_코드_명"].astype(str)
dong_df["adm_nm"] = dong_df["adm_nm"].astype(str)

df["행정동_코드_명"] = df["행정동_코드_명"].apply(
    lambda x: next((adm for adm in dong_df["adm_nm"] if x in adm), 0)
)

# df와 dong_df에서 "행정동_코드_명"과 "adm_nm"을 비교하여, 없는 행정동을 찾기
missing_dongs = dong_df[~dong_df["adm_nm"].isin(df["행정동_코드_명"])]

# missing_dongs에 있는 행정동을 df에 추가하고, 나머지 column 값은 0으로 설정
for adm_nm in missing_dongs["adm_nm"]:
    # 새로운 행을 생성
    new_row = {col: 0 for col in df.columns}  # df의 모든 열에 대해 0으로 초기화
    new_row["행정동_코드_명"] = adm_nm  # 행정동_코드_명은 해당 행정동으로 설정
    
    # df에 새로운 행을 추가
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

In [30]:
df_grouped = df.groupby("행정동_코드_명").sum().T
df_grouped = df_grouped.loc[:, df_grouped.columns != 0].T
df_grouped

,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,초등학교_수,중학교_수,고등학교_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수
행정동_코드_명,,,,,,,,,,,,,,,,,,,,
서울특별시 강남구 개포1동,570.0,15.0,15.0,0.0,0.0,15.0,0.0,15.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,435.0
서울특별시 강남구 개포2동,2025.0,45.0,45.0,0.0,0.0,135.0,75.0,60.0,15.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,750.0
서울특별시 강남구 개포4동,1725.0,45.0,60.0,0.0,0.0,105.0,0.0,30.0,15.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,735.0
서울특별시 강남구 논현1동,4500.0,60.0,225.0,0.0,60.0,630.0,0.0,15.0,0.0,0.0,0.0,0.0,15.0,120.0,0.0,0.0,0.0,0.0,45.0,345.0
서울특별시 강남구 논현2동,3600.0,120.0,240.0,0.0,75.0,435.0,15.0,15.0,15.0,0.0,60.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,15.0,495.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
서울특별시 중랑구 상봉2동,1905.0,60.0,45.0,0.0,15.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,30.0,0.0,0.0,15.0,15.0,270.0
서울특별시 중랑구 신내1동,2400.0,45.0,30.0,15.0,0.0,195.0,30.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,795.0
서울특별시 중랑구 신내2동,1560.0,90.0,45.0,0.0,0.0,180.0,45.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315.0


In [32]:
df_grouped.to_excel("접객시설.xlsx")

In [36]:
# 데이터 불러오기
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df = pd.read_csv("서울시 상권분석서비스(집객시설-행정동).csv", encoding="EUC-KR", encoding_errors="replace", low_memory=False)
df.drop(["기준_년분기_코드","행정동_코드"], axis=1, inplace=True)

dong_df = pd.read_excel("dong_info.xlsx")

df["행정동_코드_명"] = df["행정동_코드_명"].astype(str)
dong_df["adm_nm"] = dong_df["adm_nm"].astype(str)

df["행정동_코드_명"] = df["행정동_코드_명"].apply(
    lambda x: next((adm for adm in dong_df["adm_nm"] if x in adm), 0)
)

# df와 dong_df에서 "행정동_코드_명"과 "adm_nm"을 비교하여, 없는 행정동을 찾기
missing_dongs = dong_df[~dong_df["adm_nm"].isin(df["행정동_코드_명"])]

# missing_dongs에 있는 행정동을 df에 추가하고, 나머지 column 값은 0으로 설정
for adm_nm in missing_dongs["adm_nm"]:
    # 새로운 행을 생성
    new_row = {col: 0 for col in df.columns}  # df의 모든 열에 대해 0으로 초기화
    new_row["행정동_코드_명"] = adm_nm  # 행정동_코드_명은 해당 행정동으로 설정
    
    # df에 새로운 행을 추가
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

df_grouped = df.groupby("행정동_코드_명").sum().T
df_grouped = df_grouped.loc[:, df_grouped.columns != 0].T
print(df_grouped)

df_grouped.to_excel("접객시설.xlsx")

                집객시설_수  관공서_수   은행_수  종합병원_수  일반_병원_수   약국_수  유치원_수  초등학교_수  \
행정동_코드_명                                                                      
서울특별시 강남구 개포1동   570.0   15.0   15.0     0.0      0.0   15.0    0.0    15.0   
서울특별시 강남구 개포2동  2025.0   45.0   45.0     0.0      0.0  135.0   75.0    60.0   
서울특별시 강남구 개포4동  1725.0   45.0   60.0     0.0      0.0  105.0    0.0    30.0   
서울특별시 강남구 논현1동  4500.0   60.0  225.0     0.0     60.0  630.0    0.0    15.0   
서울특별시 강남구 논현2동  3600.0  120.0  240.0     0.0     75.0  435.0   15.0    15.0   
...                ...    ...    ...     ...      ...    ...    ...     ...   
서울특별시 중랑구 상봉2동  1905.0   60.0   45.0     0.0     15.0  150.0    0.0     0.0   
서울특별시 중랑구 신내1동  2400.0   45.0   30.0    15.0      0.0  195.0   30.0    30.0   
서울특별시 중랑구 신내2동  1560.0   90.0   45.0     0.0      0.0  180.0   45.0    60.0   
서울특별시 중랑구 중화1동  1185.0   30.0   15.0     0.0     15.0  105.0   15.0    30.0   
서울특별시 중랑구 중화2동  1845.0   30.0   45.0     0.0      0.